In [0]:
!pip install --upgrade tables
!pip install eli5
!pip instal xgboost
!pip install hyperopt

In [0]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import  mean_absolute_error as mae
from sklearn.model_selection import  cross_val_score, KFold
import eli5
from eli5.sklearn import PermutationImportance
import xgboost as xgb
from hyperopt import  hp, fmin, tpe, STATUS_OK

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/part2/dw_part2'


/content/drive/My Drive/Colab Notebooks/dw_matrix/part2/dw_part2


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
SUFFIX='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):
    continue
  if SUFFIX in feat: continue
  else: df[feat+SUFFIX]=df[feat].factorize()[0]
  

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0] ) )
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(" ","") ) )

In [0]:
chosen_features=['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

In [0]:
def run_model(model,features):
  X_cat= df[features].values
  y_cat=df['price_value'].values
  scores=cross_val_score(model,X_cat,y_cat,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [0]:
xgbparams={
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

In [13]:
xgb_model=xgb.XGBRegressor(**xgbparams)
run_model(xgb_model,chosen_features)

[19:28:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:28:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:28:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

Hyperopt


In [16]:
def obj_func(params):
  print("training with params: ")
  print(params)
  mean_mae, std_score = run_model(xgb.XGBRegressor(**params),chosen_features)
  return {'loss':np.abs(mean_mae),"status": STATUS_OK}

XGB_params={
    'learning_rate': hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth': hp.choice('max_depth', np.arange(4,16,1,dtype=int)),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'calsample_bytree': hp.quniform('calsample_bytree',0.5,1,0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed': 0
}

best = fmin(obj_func,XGB_params,algo=tpe.suggest,max_evals=25)

training with params: 
{'calsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
training with params: 
{'calsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
training with params: 
{'calsample_bytree': 0.9, 'learning_rate': 0.15000000000000002, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
training with params: 
{'calsample_bytree': 0.75, 'learning_rate': 0.15000000000000002, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
training with params: 
{'calsample_bytree': 0.9, 'learning_rate': 0.15000000000000002, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
training with params: 
{'calsample_bytree': 0.75, 'le

In [17]:
best

{'calsample_bytree': 0.75,
 'learning_rate': 2,
 'max_depth': 7,
 'subsample': 0.9}